In [1]:
import pandas as pd
import PyPDF2
import fitz
from io import StringIO
import shutil
import os

In [2]:
# This script takes the same annotations in one document of reference

doc = fitz.open('document_with_annotations.pdf')

rect_lists = [[] for i in range(4)] 

for page in doc:
    annots = page.annots()
    for i, annot in enumerate(annots):
        rects = annot.rect
        for rect in rects:
            rect_lists[i].append(rect)

#for rect_list in rect_lists:
    #print(rect_list)


In [3]:
# Here we iterate every document in one folder, draw the annotations and save with different name in another folder

SOURCE_FOLDER = 'SOURCE_FOLDER'
DEST_FOLDER = 'DEST_FOLDER'

for filename in os.listdir(SOURCE_FOLDER):
    doc2 = fitz.open(os.path.join(SOURCE_FOLDER, filename))
    page = doc2[0]
    for rect_list in rect_lists:    
        page.add_rect_annot(rect_list)
    doc2.save(os.path.join(DEST_FOLDER, 'CAPT_' + filename))
    doc2.close()

In [4]:

for root, dirs, files in os.walk(DEST_FOLDER):
    for file in files:
        if file.startswith("CAPT_"):
            path = os.path.join(root, file)
            file_name = file.replace("CAPT_", "")
            if os.path.exists(os.path.join(SOURCE_FOLDER, file_name)):
                os.remove(os.path.join(SOURCE_FOLDER, file_name))